In [1]:
from class_PDC import PDC
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from drawnow import drawnow
import importlib
import threading
import time
import socket

# test pdc class

In [2]:
#test if host name and port is valid
resp  = socket.getaddrinfo('mundomain.arlab1.xyz', 5714)
print('Family: {}'.format(resp[0][0]))
print('Canonname:{}'.format(resp[0][4]))


Family: 2
Canonname:('44.240.96.177', 5714)


In [3]:
#p = PDC("192.168.10.120",port=5714)
p = PDC("pdc.gl-inet.com",port=5714)
p.connect()
p.start_reading()
#time.sleep(2)
#p.stop_reading()
#p.disconnect()

In [4]:
p.start_streaming_server()

starting up on localhost port 5000


In [5]:
p.dataframes_read

,TIME,FRACSEC,STN,IDCODE,DATA_VALID,PMU_ERROR,PMU_SYNC,DATA_SORTING,TRIGGER_DETECTED,CONFIG_CHANGED,...,PHASOR5_CHNAM,PHASOR5_UNIT,PHASOR5_MAG,PHASOR5_ANG,PHASOR6_CHNAM,PHASOR6_UNIT,PHASOR6_MAG,PHASOR6_ANG,FREQ,DFREQ
EPOCH_TIME,,,,,,,,,,,,,,,,,,,,,
1.677782e+09,2023-03-02 18:41:29,0.4,OPPMU_WTPARK,4,True,False,False,timestamp,False,False,...,Phasor4,amp,70.693726,-22.544052,Phasor5,amp,70.693726,-142.544054,60.060001,0.000000e+00
1.677782e+09,2023-03-02 18:41:29,0.4,OPPMU_GPO_G2,2,True,False,False,timestamp,False,False,...,Phasor4,amp,583.463928,127.492229,Phasor5,amp,583.463928,7.492214,60.060001,7.275958e-12
1.677782e+09,2023-03-02 18:41:29,0.4,OPPMU_GPO_G1,1,True,False,False,timestamp,False,False,...,Phasor4,amp,1196.002197,120.246560,Phasor5,amp,1196.002197,-119.753420,60.060001,0.000000e+00
1.677782e+09,2023-03-02 18:41:29,0.5,OPPMU_WTPARK,4,True,False,False,timestamp,False,False,...,Phasor4,amp,70.693726,-22.544052,Phasor5,amp,70.693726,-142.544054,60.060001,0.000000e+00
1.677782e+09,2023-03-02 18:41:29,0.5,OPPMU_GPO_G2,2,True,False,False,timestamp,False,False,...,Phasor4,amp,583.463928,127.492229,Phasor5,amp,583.463928,7.492214,60.060001,7.275958e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.677783e+09,2023-03-02 18:41:52,0.4,OPPMU_GPO_G2,2,True,False,False,timestamp,False,False,...,Phasor4,amp,583.463928,127.492229,Phasor5,amp,583.463928,7.492214,60.060001,7.275958e-12
1.677783e+09,2023-03-02 18:41:52,0.4,OPPMU_GPO_G1,1,True,False,False,timestamp,False,False,...,Phasor4,amp,1196.002197,120.246560,Phasor5,amp,1196.002197,-119.753420,60.060001,0.000000e+00
1.677783e+09,2023-03-02 18:41:52,0.5,OPPMU_WTPARK,4,True,False,False,timestamp,False,False,...,Phasor4,amp,70.693726,-22.544052,Phasor5,amp,70.693726,-142.544054,60.060001,0.000000e+00


# Stream splitter

In [ ]:
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the port
server_address = ('localhost', 5000)
print('starting up on {} port {}'.format(*server_address))
sock.bind(server_address)

# Listen for incoming connections
sock.listen(2)

while True:
    # Wait for a connection
    print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(16)
            print('received {!r}'.format(data))
            if data:
                data=p.cfg2_raw
                print('sending data back to the client')
                connection.sendall(data)
                
            else:
                print('no data from', client_address)
                break

    finally:
        # Clean up the connection
        #connection.close()#%%
        pass


In [ ]:
import socket
import threading

def start_streaming_server(port=5000):
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to the port
    server_address = ('localhost', port)
    print('starting up on {} port {}'.format(*server_address))
    sock.bind(server_address)

    # Listen for incoming connections
    sock.listen(2)
    lst_connections =[]
    lst_connections_threads =[]
    while True:
        # Wait for a connection
        print('waiting for a connection')
        #connection, client_address = sock.accept()
        lst_connections.append(sock.accept())
        client_address = lst_connections[-1][1]

        try:
            print('connection from', client_address)
            lst_connections[-1][0].sendall(p.cfg2_raw)
            time.sleep(1)

            lst_connections_threads.append(threading.Thread(target=stream_dataframe,args=(lst_connections[-1][0],p
                                                                                          .dataframe_raw)))
            print('Streaming')
            lst_connections_threads[-1].start()

        finally:
            # Clean up the connection
            #connection.close()#%%
            pass


In [ ]:
thd_streaming_server = threading.Thread(target=start_streaming_server())
thd_streaming_server.start()

In [ ]:
import multiprocessing

In [ ]:
b = list(p.cfg2_raw)

In [ ]:
a = multiprocessing.Array('i',p.cfg2_raw)

In [ ]:
bytes(a)

In [ ]:
a

In [ ]:
import socket

In [ ]:
socket.AF_INET